In [209]:
import urllib.request
from bs4 import BeautifulSoup 
import requests
import json 
import re
import pandas as pd

##info
##http://stackoverflow.com/questions/11322430/python-how-to-send-post-request

#adress => en-tete de la requete > referer > @
url = "http://base-donnees-publique.medicaments.gouv.fr/index.php"  

# get the inspect element in the search bar 
# recharge
# look for "ibuprofène"
# you will see a POST request
# parameters after input>> reseau > parametre > copy the dictionnary
data_post1 ={'page':"1", 'affliste' :"0", 'affNumero' :"0",'affNumero' :"0",'isAlphabet' :"0",'inClauseSubst' :"0",'nomSubstances' :"",'typeRecherche' :"0",'choixRecherche' :"medicament",'paginationUsed' :"0",'txtCaracteres' :"ibuprofene",'btnMedic.x' :"0",'btnMedic.y' :"0",'radLibelle' :"2",'txtCaracteresSub' :"",'radLibelleSub' :"4"}
# on peut prendre que les valeurs non nulles
data_post3={'page':"3",'choixRecherche' :"medicament", 'txtCaracteres' :"ibuprofene",'radLibelle' :"2",'radLibelleSub' :"4"}
data_post2={'page':"2",'choixRecherche' :"medicament", 'txtCaracteres' :"ibuprofene",'radLibelle' :"2",'radLibelleSub' :"4"}

r = requests.post(url, data_post1 )
print(r)

#make soup soup = BeautifulSoup(r,"html.parser")
#res=soup.find_all("td", class_="alignleft") test=  soup.find_all('caption')

#<td class="alignleft" 
  
#nb_medicament =
#molecule,nom du medoc, unite de dosage, forme secable, laboratoire.. dans un dataframe 

<Response [200]>


In [220]:
print(r.status_code, r.reason)

200 OK


In [213]:
r.text #donne la nouvelle page avec la nouvelle url mais en moche
# type(r.text ) #str
res=BeautifulSoup(r.text, "lxml") #tansform into beautiful
res_medoc =res.find_all("a",class_="standart") #les nom des medoc mais il reste  page 2 et page 3
res_medoc[0]#get one line of the drugs
str(res_medoc[0]) #cast to str
# print(type(res_medoc) )# size  
# res_medoc[5].text
# res_medoc

'<a class="standart" href="extrait.php?specid=61349237" title="Cliquez pour accéder à la fiche info et à la liste des documents de référence du médicament :  IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé">IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé\t\t\t\t</a>'

In [214]:
for i in range(20):
    try :
        print(res_medoc[i].text)
    except IndexError: 
        print("")

IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé				
IBUPROFENE ALMUS 200 mg, comprimé pelliculé				
IBUPROFENE ALMUS 400 mg, comprimé pelliculé				
IBUPROFENE ARROW 200 mg, comprimé enrobé				
IBUPROFENE ARROW 400 mg, comprimé pelliculé				
IBUPROFENE ARROW 5 %, gel				
IBUPROFENE ARROW CONSEIL 400 mg, comprimé pelliculé				
IBUPROFENE BIOGARAN 200 mg, comprimé pelliculé				
IBUPROFENE BIOGARAN 400 mg, comprimé pelliculé				
IBUPROFENE BIOGARAN CONSEIL 200 mg, comprimé pelliculé				
IBUPROFENE BIOGARAN CONSEIL 400 mg, comprimé pelliculé				
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée				
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée en flacon pressurisé				
IBUPROFENE CRISTERS 200 mg, comprimé enrobé				
IBUPROFENE CRISTERS 200 mg, comprimé pelliculé				
IBUPROFENE CRISTERS 400 mg, comprimé pelliculé				
IBUPROFENE DU SAINT BERNARD 5 %, gel pour application locale				
IBUPROFENE EG 200 mg, comprimé pelliculé				
IBUPROFENE EG 400 mg, comprimé pelliculé				
IBU

In [215]:
medoc=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[0]))
medoc.group(0)

'IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé'

In [216]:
for i in range(20): 
    try :
        final =re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%\/]+),([ \w]+)', res_medoc[i].text).group(0)  
        print(final)
    except IndexError: 
        print("")

IBUPROFENE ACTAVIS 400 mg, comprimé pelliculé
IBUPROFENE ALMUS 200 mg, comprimé pelliculé
IBUPROFENE ALMUS 400 mg, comprimé pelliculé
IBUPROFENE ARROW 200 mg, comprimé enrobé
IBUPROFENE ARROW 400 mg, comprimé pelliculé
IBUPROFENE ARROW 5 %, gel
IBUPROFENE ARROW CONSEIL 400 mg, comprimé pelliculé
IBUPROFENE BIOGARAN 200 mg, comprimé pelliculé
IBUPROFENE BIOGARAN 400 mg, comprimé pelliculé
IBUPROFENE BIOGARAN CONSEIL 200 mg, comprimé pelliculé
IBUPROFENE BIOGARAN CONSEIL 400 mg, comprimé pelliculé
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée
IBUPROFENE CHEMINEAU 5%, solution pour application cutanée en flacon pressurisé
IBUPROFENE CRISTERS 200 mg, comprimé enrobé
IBUPROFENE CRISTERS 200 mg, comprimé pelliculé
IBUPROFENE CRISTERS 400 mg, comprimé pelliculé
IBUPROFENE DU SAINT BERNARD 5 %, gel pour application locale
IBUPROFENE EG 200 mg, comprimé pelliculé
IBUPROFENE EG 400 mg, comprimé pelliculé
IBUPROFENE EVOLUGEN 400 mg, comprimé pelliculé


In [217]:
# page 1+ 3
L=[]
# d = {'col1': ts1, 'col2': ts2}
for i in range(20):
    tmp=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[i])).group(0)
    len = tmp.find(" ")
    mol=tmp[:len]
    nom =re.search( ' ([A-Z ]+)',tmp).group(0)
    qte=re.search( '(\d+) ?([\w%]+)',tmp).group(0)
    type=re.search( ',([ \w]+)',tmp).group(0)[1:]
    l = {'nom molecule': mol, 'nom': nom,"quantité":qte,"type":type}
    L.append(l)

In [143]:
# page 2
L=[]
# d = {'col1': ts1, 'col2': ts2}
for i in range(20):
    try:
        tmp=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%\/]+).+,([ \w]+)',res_medoc[i].text).group(0)
        len = tmp.find(" ")
        mol=tmp[:len]
        nom =re.search( ' ([A-Z ]+)',tmp).group(0)
        qte=re.search( '(\d+) ?([\w%\/]+).+',tmp).group(0)
        type=re.search( ',([ \w]+)',tmp).group(0)[1:]
        l = {'nom molecule': mol, 'nom': nom,"quantité":qte,"type":type}
    except AttributeError:
        tmp=(re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%\/]+) ([ \w]+)', res_medoc[i].text).group(0) )
        len = tmp.find(" ")
        mol=tmp[:len]
        nom =re.search( ' ([A-Z ]+)',tmp).group(0)
        qte=re.search( '(\d+) ?([\w%\/]+)',tmp).group(0)
        type=re.search( '([%]) ([ \w]+)',tmp).group(0)[1:]
        l = {'nom molecule': mol, 'nom': nom,"quantité":qte,"type":type}
    L.append(l)

In [144]:
df=pd.DataFrame(L)
df

,nom,nom molecule,quantité,type
0,ISOMED,IBUPROFENE,"200 mg, comprimé enrobé",comprimé enrobé
1,MYLAN,IBUPROFENE,"20 mg/ml ENFANTS ET NOURRISSONS sans sucre, su...",suspension buvable édulcorée au maltitol et à...
2,MYLAN,IBUPROFENE,"200 mg, comprimé enrobé",comprimé enrobé
3,MYLAN,IBUPROFENE,"400 mg, comprimé effervescent",comprimé effervescent
4,MYLAN,IBUPROFENE,"400 mg, comprimé pelliculé",comprimé pelliculé
5,MYLAN,IBUPROFENE,5 %,Solution pour pulvérisation cutanée
6,MYLAN CONSEIL,IBUPROFENE,"400 mg, comprimé pelliculé",comprimé pelliculé
7,NEPENTHES,IBUPROFENE,"200 mg, comprimé pelliculé",comprimé pelliculé
8,NEPENTHES,IBUPROFENE,"400 mg, comprimé pelliculé",comprimé pelliculé
9,PHR LAB,IBUPROFENE,"400 mg, comprimé pelliculé",comprimé pelliculé


In [59]:
#re.search( '(\d+)',tmp.group(0))
type(tmp.group(0))

str

In [114]:
re.search( '(\d+) ?([\w%]+)','IBUPROFENE EG 200 mg, comprimé pelliculé')

<_sre.SRE_Match object; span=(14, 20), match='200 mg'>

In [102]:
re.search( ' ([A-Z ]+)','IBUPROFENE CHEMINEAU 5%, solution pour application cutanée')

<_sre.SRE_Match object; span=(10, 21), match=' CHEMINEAU '>

In [103]:
re.search( ',([ \w]+)','IBUPROFENE CHEMINEAU 5%, solution pour application cutanée')

<_sre.SRE_Match object; span=(23, 58), match=', solution pour application cutanée'>

In [65]:
tmp.group(0).find(" ")

10

In [66]:
tmp.group(0)[:10]

'IBUPROFENE'

In [19]:
last_page= res.find_all("div", class_="navBarGauche") [0].text.replace("\xa0","")
print(last_page)

Page 1 / 3


In [20]:
last_page.find('/')

7

In [40]:
num_last_page= int(last_page[last_page.find('/')+1:].strip())

In [41]:
list_dico=[]

In [218]:
def get_df(data_post):
    r = requests.post(url, data_post )
    res=BeautifulSoup(r.text, "lxml") #tansform into beautiful
    res_medoc =res.find_all("a",class_="standart") #les nom des medoc mais il reste  page 2 et page 3
    '''
    res_medoc[0]#get one line of the drugs
    str(res_medoc[0]) #cast to str
    len(res_medoc)# size  
    '''

    L=[]
    # d = {'col1': ts1, 'col2': ts2}
    for i in range(20):
        tmp=re.search('IBUPROFENE ([A-Z ]+) (\d+) ?([\w%]+),([ \w]+)', str(res_medoc[i])).group(0)
        len = tmp.find(" ")
        mol=tmp[:len]
        nom =re.search( ' ([A-Z ]+)',tmp).group(0)
        qte=re.search( '(\d+) ?([\w%]+)',tmp).group(0)
        type=re.search( ',([ \w]+)',tmp).group(0)[1:]
        l = {'nom molecule': mol, 'nom': nom,"quantité":qte,"type":type}
        L.append(l)

    #pour page 1
    df=pd.DataFrame(L)
    return df

In [221]:
df1= get_df(data_post1)
